# An user guid of `keras-unet-collection`

In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
print('TensorFlow {}; Keras {}'.format(tf.__version__, keras.__version__))

TensorFlow 2.3.0; Keras 2.4.0


### Step 1: importing `models` from `keras_unet`

In [3]:
from keras_unet_collection import models

### Step 2: defining your hyper-parameters

**Example 1**: A binary classification U-net with:

1. Four down- and upsampliung levels.

2. Two convolutional layers per downsampling level.

3. One convolutional layer (after concatenation) per upsamling level.

2. ReLU activcation, batch normalization.

3. Downsampling through Maxpooling.

4. Upsampling through reflective padding.

In [4]:
unet = models.unet_2d((64, 64, 3), [64, 128, 256, 512, 1024], n_labels=2,
                      stack_num_down=2, stack_num_up=1,
                      activation='ReLU', output_activation='Softmax', 
                      batch_norm=True, pool=True, unpool=True, name='unet')

**Example 2**: An attention-Unet for single target regression with:

1. three down- and upsampliung levels.

2. Two convolutional layers per downsampling level.

3. Two convolutional layers (after concatenation) per upsamling level.

2. ReLU activcation, batch normalization.

3. Additive attention, ReLU attention activation
        
4. downsampling through stride convolutional layers.

5. Upsampling through transpose covolutional layers.   
        

In [5]:
att_unet = models.att_unet_2d((64, 64, 3), [64, 128, 256, 512], n_labels=1,
                              stack_num_down=2, stack_num_up=2,
                              activation='ReLU', atten_activation='ReLU', attention='add', output_activation=None, 
                              batch_norm=True, pool=False, unpool=False, name='att-unet')

**Example 3**: An U-net++ for three-label classification with:

1. three down- and upsampliung levels.

2. Two convolutional layers per downsampling level.

3. Two convolutional layers (after concatenation) per upsamling level.

2. LeakyReLU activcation, no batch normalization.
        
3. Downsampling through Maxpooling.

4. Upsampling through transpose covolutional layers.  

In [6]:
xnet = models.unet_plus_2d((64, 64, 3), [64, 128, 256, 512], n_labels=3,
                           stack_num_down=2, stack_num_up=2,
                           activation='LeakyReLU', output_activation='Softmax', 
                           batch_norm=False, pool=True, unpool=False, name='xnet')

**Example 4**: Going deeper --- dding residual blocks into the Example 3 U-net++

In [7]:
res_xnet = models.res_unet_plus_2d((64, 64, 3), [64, 128, 256, 512], n_labels=3,
                                   stack_num=2, res_num=2,
                                   activation='LeakyReLU', output_activation='Softmax', 
                                   batch_norm=False, pool=True, unpool=False, name='res_xnet')